In [11]:
from sympy import *
from itertools import *
from MRG import *
d = 1239

In [12]:
# Comprobamos que es libre de cuadrados
all([i == 1 for i in factorint(d).values()])

True

In [13]:
# Cota de Minkowski
# Discriminante del cuerpo
def discriminante(d):
    if d%4 != 1:
        return 4*d
    else:
        return d

In [14]:
def minkowski(d):
    # Definición de s y t
    if d > 0:
        s = 2
        t = 0
    else:
        s = 0
        t = 1
    n = s + 2*t
    
    cota = (4/float(pi))**t * factorial(n)/(n**n) * sqrt(discriminante(d))
    return float(cota)

In [15]:
minkowski(1239)

35.199431813596085

In [16]:
# Buscamos los primos menores que la cota
primosdebajo = [x for x in sieve.primerange(0,minkowski(1239))]
primosdebajo

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31]

In [17]:
# Nos quedamos sólo con los primos que ramifican
def ramifica(p):
    irr = irr_e(d)
    solucionesirr = [x for x in xrange(p) if irr(x)%p==0]
    return (len(solucionesirr) != 0)
    
primosramifican = filter(ramifica, primosdebajo)
primosramifican

[2, 3, 5, 7, 13, 17, 19]

In [18]:
primosqueramificanbajolacota(d)

[2, 3, 5, 7, 13, 17, 19]

In [19]:
idealesGeneradores(d)

[[2, -1 + sqrt(1239)],
 [3, sqrt(1239)],
 [5, -2 + sqrt(1239)],
 [7, sqrt(1239)],
 [13, -2 + sqrt(1239)],
 [17, -7 + sqrt(1239)],
 [19, -2 + sqrt(1239)]]

In [20]:
# Los generadores son los ideales con la norma de los primos que ramifican.
# Podemos coger sólo el primero porque si hay otro entonces serán inversos
ideales = map(lambda p: idealesDivisores(p,d)[0], primosramifican)
ideales

[[2, -1 + sqrt(1239)],
 [3, sqrt(1239)],
 [5, -2 + sqrt(1239)],
 [7, sqrt(1239)],
 [13, -2 + sqrt(1239)],
 [17, -7 + sqrt(1239)],
 [19, -2 + sqrt(1239)]]

In [23]:
lista = refinaGeneradores(ideales,d)

El ideal 0 ésimo es inverso del ideal 3 ésimo.
El ideal 1 ésimo es inverso del ideal 5 ésimo.
El ideal 2 ésimo es inverso del ideal 4 ésimo.


In [21]:
def esPrinc(ideal,d):
    n = normaIdeal(ideal,d)
    l = connorma(n,d) + connorma(-n,d)
    return any([pertenece(u,ideal,d) for u in l])

In [22]:
# Primer refinamiento
lista = filter(lambda i: not esPrinc(i,d), ideales)
lista

[[2, -1 + sqrt(1239)],
 [3, sqrt(1239)],
 [5, -2 + sqrt(1239)],
 [7, sqrt(1239)],
 [13, -2 + sqrt(1239)],
 [17, -7 + sqrt(1239)],
 [19, -2 + sqrt(1239)]]

In [96]:
# Segundo refinamiento: quitar inversos
def quitaInversos(lista,d):
    i = 0
    while i < len(lista):
        j = i+1
        while j < len(lista):
            print i,j
            if esPrinc(productodos(lista[i],lista[j],d),d):
                print "Inversos: ",i,j
                del lista[j]
            else:
                j = j+1
        i = i+1

In [97]:
quitaInversos(lista,d)

0 1
0 2
0 3
Inversos:  0 3
0 3
0 4
0 5
1 2
1 3
1 4
1 5
Inversos:  1 5
2 3
2 4
Inversos:  2 4


In [17]:
lista

[[2, -1 + sqrt(1239)],
 [3, sqrt(1239)],
 [5, -2 + sqrt(1239)],
 [13, -2 + sqrt(1239)]]

In [101]:
def orden(ideal, d):
    n = 1
    i = ideal
    while not esPrinc(i,d):
        n = n+1
        i = productodos(i,ideal,d)
    return n

In [13]:
orders = map(lambda i: (i,orden(i, d)), lista)
orders

NameError: name 'lista' is not defined

In [19]:
ordenLista(lista,d)

[([2, -1 + sqrt(1239)], 2),
 ([3, sqrt(1239)], 2),
 ([5, -2 + sqrt(1239)], 4),
 ([13, -2 + sqrt(1239)], 4)]

https://math.stackexchange.com/questions/1509214/express-an-abelian-group-given-as-finite-generators-and-their-relations-as-a-dir

In [135]:
relationlist = map(lambda (a,b): b, orders)
relationmatrix = [
    [x if i==j else 0 for j in range(len(relationlist))] 
    for (i,x) in list(enumerate(relationlist))]

In [136]:
relationmatrix

[[2, 0, 0, 0], [0, 2, 0, 0], [0, 0, 4, 0], [0, 0, 0, 4]]

In [105]:
def calculaRelacion(rel, generadores, d):
    resultado = [1]
    for i in xrange(len(rel)):
        for _ in xrange(rel[i]):
            resultado = productodos(resultado, generadores[i], d)
    return resultado

In [106]:
def esNuevaRelacion(rel, matriz, generadores, d):
    for relacion in matriz:
        if all([a >= b for (a,b) in zip(rel,relacion)]):
            return False
    
    if esPrinc(calculaRelacion(rel, generadores, d),d):
        return True
    
    return False

In [108]:
def sumasposibles(r, n, t, acc=[]):
    if t == 0:
        if n == 0:
            yield acc
        return
    for x in r:
        if x > n:
            break
        for lst in sumasposibles(r, n-x, t-1, acc + [x]):
            yield lst

In [111]:
def encuentraRelaciones(matriz, generadores, d):
    t = len(generadores)
    n = 3
    alguncaso = True
    
    while alguncaso:
        print "Testing with n =",n
        alguncaso = False
        for suma in sumasposibles(range(n+1),n,t):
            if esNuevaRelacion(suma, matriz, generadores, d):
                matriz = [suma] + matriz
                print suma
                alguncaso = True
        n = n+1
    
    return matriz

In [137]:
final = encuentraRelaciones(relationmatrix, lista, d)

Testing with n = 3
[0, 1, 1, 1]
Testing with n = 4
[0, 0, 2, 2]
[1, 1, 0, 2]
[1, 1, 2, 0]
Testing with n = 5
[1, 0, 1, 3]
[1, 0, 3, 1]
Testing with n = 6


In [138]:
final2 = final

In [148]:
final2

[[1, 0, 3, 1],
 [0, 1, -1, -1],
 [0, 1, -3, 1],
 [0, 0, -6, -2],
 [0, 0, -2, 2],
 [0, 0, 2, 2],
 [0, 1, 1, 1],
 [0, 2, 0, 0],
 [0, 0, 4, 0],
 [0, 0, 0, 4]]

In [155]:
def reduceRelaciones(matriz):
    n = len(matriz)
    m = len(matriz[0])
    
    for step in range(m):

        # mientras que la matriz no esté reducida
        while any([fila[step] != 0 for fila in matriz[(step+1):]]):

            # Ordena la matriz por valor absoluto, dejando ceros al final
            matriz[step:] = sorted(matriz[step:],
                key=lambda x: abs(x[step]) if x[step] != 0 else float("inf"))

            # Y simplifica por filas
            pivote = matriz[step][step]
            for i in range(step+1,n):
                coc = matriz[i][step] / pivote        
                for j in range(m):
                    matriz[i][j] = matriz[i][j] - matriz[step][j] * coc

    return matriz

In [156]:
reduceRelaciones(final2)

[[1, 0, 3, 1],
 [0, 1, -1, -1],
 [0, 0, -2, 2],
 [0, 0, 0, 4],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0],
 [0, 0, 0, 0]]